### Model to Streamlit

In [15]:
import sys
sys.path.insert(0, '../')


In [16]:
import file_paths
from basic_pitch_module.basic_pitch.inference import predict
from data_processing.midi2CP import CP
import pickle
import miditoolkit
import io
from data_processing.extract import main


In [17]:
## Test
from data_processing.extract import main


In [18]:
midi_data = predict('../temp/test/1.mp3')[1]

midi_stream = io.BytesIO()
midi_data.write(midi_stream)
midi_stream.seek(0)
midi_data = miditoolkit.midi.parser.MidiFile(file = midi_stream)


Predicting MIDI for ../temp/test/1.mp3...


In [19]:
midi_data


ticks per beat: 220
max tick: 198698
tempo changes: 1
time sig: 1
key sig: 0
markers: 0
lyrics: False
instruments: 1

In [20]:
#with open(file_paths.cp_dict, 'rb') as f:
with open('/Users/d.haverkort/code/HugoA45/music_project/music_project/resources/CP.pkl', 'rb') as f:
    e2w, w2e = pickle.load(f)

    compact_classes = ['Bar', 'Position', 'Pitch', 'Duration']
    pad_CP = [e2w[subclass][f"{subclass} <PAD>"] for subclass in compact_classes]

    # preprocess input file
    CP_model = CP('/Users/d.haverkort/code/HugoA45/music_project/music_project/resources/CP.pkl')

    events, tokens = CP_model.prepare_data(midi_data, 512)


In [21]:
tokens.shape


(11, 512, 4)

In [22]:
from bert.load_midi_bert import load_model


In [23]:
dict_path = '/Users/d.haverkort/code/HugoA45/music_project/music_project/resources/CP.pkl'
ckpt_path = '/Users/d.haverkort/code/HugoA45/music_project/music_project/resources/pretrain_model.ckpt'
midi_bert = load_model(dict_path, ckpt_path)


Loading Dictionary


In [24]:
from predict.model_prediction import prediction

unts = prediction(tokens, midi_bert, task='composer_classification')


In [25]:
unts.value_counts()/len(unts)


Joe Hisaishi (Contemporary)        0.727273
Ryuichy Sakamoto (Contemporary)    0.181818
Bethel Music (Religious)           0.090909
dtype: float64

In [28]:
import os
current_directory = os.path.dirname(os.path.abspath(__file__))
dict_path = os.path.join(current_directory, 'resources', 'CP.pkl')
ckpt_path = os.path.join(current_directory, 'resources', 'pretrain_model.ckpt')


NameError: name '__file__' is not defined

In [27]:
dict_path


'/Users/d.haverkort/code/HugoA45/music_project/music_project/notebooks/music_project/resources/CP.pkl'

In [ ]:
# 2.1 Website Function: typewriter - slow down typing speed
# def typewriter(text: str, speed=10):
#     tokens = text.split()
#     container = st.empty()
#     for index in range(len(tokens) + 1):
#         curr_full_text = " ".join(tokens[:index])
#         container.markdown(curr_full_text)
#         time.sleep(1 / speed)
